In [1]:
import numpy as np 
import pandas as pd 
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
dint_encoder = {
  'AA': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'AC': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'AG': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'AT': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'CA': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'CC': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'CG': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'CT': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  'GA': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  'GC': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  'GG': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  'GT': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  'TA': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  'TC': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  'TG': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  'TT': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
}
def __dinucleotide_encode(seq):
  ohe = []
  for idx in range(len(seq) - 1):
    ohe += dint_encoder[seq[idx : idx + 2]]
  return ohe

# skips 0; PAM starts at 1, last int inclusive
def __get_dinucleotide_nms(start_pos, end_pos):
    nms = []# Convert lists to sets
set1 = set(list1)
set2 = set(list2)

# Find the intersection of the two sets
similar_elements = set1.intersection(set2)

# Convert the set back to a list if needed
similar_elements_list = list(similar_elements)

print(similar_elements_list)
    dints = sorted(list(dint_encoder.keys()))
    for pos in range(start_pos, end_pos+1):
        if pos == 0:
            continue
        for dint in dints:
            nms.append('%s%s' % (dint, pos))
    return nms

# for context, skips middle 
def encode_7mer_excluding_middle(seq):
    assert len(seq) == 7, "Sequence must be exactly 7 nucleotides long"
    ohe = []
    for idx in range(2): 
        ohe += dint_encoder[seq[idx: idx + 2]]
    
    for idx in range(4, 6):  
        ohe += dint_encoder[seq[idx: idx + 2]]
    return ohe

# SPECIFIC FOR CONTEXT (7 NUCLEOTIDES)
def get_dinucleotide_nms_context():
    dints = sorted(list(dint_encoder.keys()))
    feature_names = []
    for pos in [1, 2]:
        for dint in dints:
            feature_names.append(f'{dint}{pos}')
    for pos in [5, 6]:
        for dint in dints:
            feature_names.append(f'{dint}{pos}')
    return feature_names


def generate_feature_mx(df_ed):
    feature_records = []
    
    nucleotides = ['A', 'C', 'G', 'T']
    

    for _, row in df_ed.iterrows():
        features = {
            'eff': row['eff'],
            'SGN_Strand': row['strand'],
            'Editing_Position': -1 * row['pos']
        }

        unique_editing_positions = set(df_ed['pos'].tolist())
        positions = [0 for x in unique_editing_positions]
        editing_position = row['pos']

        for index, position in enumerate(unique_editing_positions):
            if position != editing_position:
                features[f'Editing_P{position}'] = positions[index]
            else:
                positions[index] = 1
                features[f'Editing_P{position}'] = positions[index]


        seq_nb = row['context']
        for k, v in enumerate(seq_nb):
            if k == 3:
                continue
            for nt in nucleotides:
                features[f'Context_P{str(k - 3)}_{nt}'] = int(nt == v)

        features['Editing_mt'] = mt.Tm_Wallace(seq_nb) / 100
        sgn = row['target']
        for k in range(0, 30, 4):
            seq_nb2 = Seq(sgn[k:k+10])
            mt_w = mt.Tm_Wallace(seq_nb)
            features['SGN_mt_w_'+str(k)] = mt_w/100

        # Processing target sequence
        for k, nt in enumerate(sgn, -len(sgn)):
            pos_label = k + 10 if k + 10 >= 0 else k + 11
            for bs in nucleotides:
                features[f'SGN_P{pos_label}_{bs}'] = int(bs == nt)

        # # dincucleotide
        # context_ohe = encode_7mer_excluding_middle(seq_nb)
        # context_nm = get_dinucleotide_nms_context()
        # for ohe in zip(context_nm,context_ohe):
        #     features["di_context_"+ohe[0]] = ohe[1]
        
        # # target
        target_no_pam = row['target_no_pam']
        # dseq = __dinucleotide_encode(target_no_pam)
        # dnms = __get_dinucleotide_nms(-31,-1)
        # for ohe in zip(dnms,dseq):
        #     features["di_target_no_pam_"+ohe[0]] = ohe[1]

        # # pam 
        # pam = row['pam']
        # pamseq = __dinucleotide_encode(pam)
        # pamnms = __get_dinucleotide_nms(1,10)
        # for ohe in zip(pamnms,pamseq):
        #     features["di_pam_"+ohe[0]] = ohe[1]

        # nucleotide counts
        counts = len(target_no_pam)
        nuc_content = [
            target_no_pam.count('A')/counts,
            target_no_pam.count('C')/counts,
            target_no_pam.count('G')/counts,
            target_no_pam.count('T')/counts,
            (target_no_pam.count('G') + target_no_pam.count('C'))/counts
          ]
        nuc_names = ['A', 'C', 'G', 'T', 'GC']

        for nc in zip(nuc_names, nuc_content):
            features[nc[0]+'content'] = nc[1]

        feature_records.append(features)

    df_stat_feat = pd.DataFrame(feature_records)
    return df_stat_feat

IndentationError: unexpected indent (4072002506.py, line 42)

In [3]:
import pandas as pd
import numpy as np
from feature_matrix_generator import generate_feature_mx
df_editing = pd.read_csv("../FH_lenti_editing_summary_processed.csv")
spacer_length = 23
df_editing['target_no_pam'] = df_editing['target'].apply(lambda x: x[:-10])
df_editing['pam'] = df_editing['target'].apply(lambda x: x[-10:])
df_editing = df_editing[df_editing.spacer_length == spacer_length]


generated_mx = generate_feature_mx(df_editing)

In [5]:
generated_mx.to_csv("test.csv", index = False)

In [24]:
feature_list = ['SGN_Strand', 'Editing_Position', 'SGN_P2_T', 'GCcontent', 'Tcontent', 'Gcontent', 'SGN_P1_G', 'SGN_P-23_G', 'SGN_P-22_G', 'SGN_P8_T', 'Acontent', 'Ccontent', 'SGN_P-17_C', 'Context_P-1_T', 'SGN_P-9_G', 'SGN_P-6_C', 'SGN_P2_G', 'SGN_P-3_A', 'SGN_P-1_T', 'SGN_P-5_C', 'SGN_P-10_C', 'SGN_P-21_C', 'SGN_P-10_A', 'Context_P1_C', 'SGN_P-20_A', 'Context_P-2_A', 'SGN_P-1_G', 'Editing_P-10', 'Context_P-2_G']

In [25]:
def update_dataframe(df, feature_list):
    """
    Takes in a processed dataframe and a list of features. Will create columns from specified paired features and then remove the original columns from the dataframe. Ignores the last feature if an odd number of features are provided.
    
    Parameters:
    df (pandas.DataFrame): The dataframe to be updated.
    feature_list (list): List of column names in the dataframe to be used for creating paired features.
    
    Returns:
    pandas.DataFrame: The updated dataframe with new paired features and without the original features used for pairing.
    """
    bin_dict = {(0, 0): 0, (0, 1): 1, (1, 0): 2, (1, 1): 3}

    unwanted_list = ['SGN_Strand', 'Editing_Position', 'Acontent', 'Tcontent', 'Ccontent', 'Gcontent', 'GCcontent', 'Editing_mt']
    begin = 'SGN_mt_w_'
    
    feature_list = [feature for feature in feature_list if feature not in unwanted_list and not feature.startswith(begin)]
    
    for i in range(0, len(feature_list) - len(feature_list) % 2, 2):
        feature1 = feature_list[i]
        feature2 = feature_list[i + 1]
        
        new_column_name = f"paired_{feature1}_{feature2}"
        
        df[new_column_name] = df[[feature1, feature2]].apply(lambda row: bin_dict[(row[feature1], row[feature2])], axis=1)
        
        df.drop(columns=[feature1, feature2], inplace=True)

    return df


In [26]:
df = update_dataframe(generated_mx, feature_list)
df

,eff,SGN_Strand,Editing_Position,Editing_P-25,Editing_P-24,Editing_P-23,Editing_P-22,Editing_P-21,Editing_P-20,Editing_P-19,...,paired_SGN_P-23_G_SGN_P-22_G,paired_SGN_P8_T_SGN_P-17_C,paired_Context_P-1_T_SGN_P-9_G,paired_SGN_P-6_C_SGN_P2_G,paired_SGN_P-3_A_SGN_P-1_T,paired_SGN_P-5_C_SGN_P-10_C,paired_SGN_P-21_C_SGN_P-10_A,paired_Context_P1_C_SGN_P-20_A,paired_Context_P-2_A_SGN_P-1_G,paired_Editing_P-10_Context_P-2_G
0,0.004985,0.0,25,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,2,0
1,0.001656,1.0,25,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,2,0
2,0.039767,0.0,25,1,0,0,0,0,0,0,...,0,2,0,0,1,0,1,3,0,0
3,0.018538,1.0,25,1,0,0,0,0,0,0,...,0,2,0,0,1,0,1,3,0,0
4,0.033120,0.0,25,1,0,0,0,0,0,0,...,0,2,0,0,1,0,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2629,0.000564,1.0,1,0,0,0,0,0,0,0,...,0,0,0,2,0,2,3,3,0,0
2630,0.000130,0.0,1,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,1,0,1
2631,0.000328,0.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,2,0,1,0,0
2632,0.000202,1.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,2,0,1,0,0


In [33]:
features = ['SGN_Strand', 'Editing_Position', 'Editing_P-10.0', 'Context_P-3_C', 'Context_P-3_G', 'Context_P-1_C', 'Context_P-1_T', 'Context_P1_C', 'Context_P2_A', 'Context_P2_C', 'Context_P2_T', 'Context_P3_G', 'SGN_mt_w_12', 'SGN_mt_w_28', 'SGN_P-30_G', 'SGN_P-26_G', 'SGN_P-25_A', 'SGN_P-24_G', 'SGN_P-23_T', 'SGN_P-21_C', 'SGN_P-19_A', 'SGN_P-19_C', 'SGN_P-19_T', 'SGN_P-18_A', 'SGN_P-18_G', 'SGN_P-17_A', 'SGN_P-17_T', 'SGN_P-16_C', 'SGN_P-14_A', 'SGN_P-14_T', 'SGN_P-12_G', 'SGN_P-11_A', 'SGN_P-11_C', 'SGN_P-9_C', 'SGN_P-9_T', 'SGN_P-8_G', 'SGN_P-7_T', 'SGN_P-6_A', 'SGN_P-5_A', 'SGN_P-5_C', 'SGN_P-3_A', 'SGN_P-3_C', 'SGN_P-1_T', 'SGN_P0_A', 'SGN_P0_C', 'SGN_P1_G', 'SGN_P2_G', 'SGN_P2_T', 'SGN_P6_A', 'SGN_P6_G', 'SGN_P6_T', 'SGN_P7_T', 'SGN_P8_T', 'SGN_P9_C', 'SGN_P9_T', 'Acontent', 'Ccontent', 'Gcontent', 'Tcontent', 'GCcontent']

In [44]:
import joblib
model = joblib.load("./best_model.pkl")

df_editing = pd.read_csv("../FH_lenti_editing_summary_processed.csv")
spacer_length = 23
df_editing['target_no_pam'] = df_editing['target'].apply(lambda x: x[:-10])
df_editing['pam'] = df_editing['target'].apply(lambda x: x[-10:])
df_editing = df_editing[df_editing.spacer_length == spacer_length]
generated_mx = generate_feature_mx(df_editing)


for x in generated_mx.columns:
    if x == "Editing_P-10.0":
        print('opk')
generated_mx.rename(columns={"Editing_P-10": "Editing_P-10.0"}, inplace=True)

predictions = model.predict(generated_mx[features])



In [45]:
predictions

array([0.01489704, 0.01010012, 0.01290658, ..., 0.01053574, 0.00065119,
       0.00116845])

In [47]:
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr


import numpy as np

y_true = df_editing['eff']

r_squared = r2_score(y_true, predictions)
print(f"R-squared: {r_squared}")

# Calculate Pearson correlation
correlation, p_value = pearsonr(y_true, predictions)
print(f"Pearson correlation: {correlation}")

# Calculate Spearman correlation
spearman_corr, spearman_p_value = spearmanr(y_true, predictions)
print(f"Spearman correlation: {spearman_corr}")




R-squared: 0.0674486747371662
Pearson correlation: 0.5892043430827754
Spearman correlation: 0.6123057905222535


In [15]:
import pandas as pd 
df = pd.read_csv("./p_values_20_to_bottom.csv")

In [16]:
feature_list = ['SGN_P4_A', 'SGN_P4_C', 'SGN_P4_G', 'SGN_P4_T', 'SGN_P5_A', 'SGN_P5_C', 'SGN_P5_G', 'SGN_P5_T', 'Editing_P-1', 'Editing_P-2', 'Editing_P-3', 'Editing_P-4', 'Editing_P-25', 'Editing_P-24', 'Editing_P-5', 'Editing_P-23', 'Editing_P-8', 'SGN_P-12_A', 'SGN_P-4_A', 'SGN_P-24_T', 'SGN_P-21_A', 'Editing_P-17', 'SGN_P6_C', 'Editing_P-22', 'SGN_P-21_T', 'SGN_P-28_A', 'SGN_P-27_T', 'SGN_P8_A', 'SGN_P-26_T', 'SGN_P-4_T', 'SGN_P-1_C', 'SGN_P-29_A', 'Editing_P-21', 'SGN_P-13_G', 'SGN_P-22_T', 'SGN_P-16_A', 'SGN_P-26_C', 'SGN_P-8_T', 'Editing_mt', 'Editing_P-20', 'SGN_P-23_C', 'SGN_P-15_T', 'SGN_P-20_T', 'Editing_P-13', 'Editing_P-6', 'SGN_mt_w_0', 'SGN_P-5_G', 'SGN_P8_C', 'SGN_P-28_G', 'SGN_P-1_G', 'Editing_P-19', 'SGN_P2_A', 'SGN_mt_w_16', 'SGN_P-12_C', 'SGN_P-29_T', 'SGN_P-12_T', 'SGN_P-25_T', 'SGN_P-25_G', 'SGN_P-30_T', 'Context_P-1_A', 'SGN_P-14_G', 'SGN_P-16_G', 'SGN_P-2_T', 'SGN_mt_w_4', 'SGN_mt_w_24', 'SGN_P3_A', 'SGN_P-4_C', 'SGN_P-13_T', 'SGN_P-5_T', 'SGN_P-24_C', 'SGN_P-25_C', 'SGN_P-27_A', 'SGN_P-23_A', 'SGN_P-30_C', 'Editing_P-9', 'SGN_P1_T', 'SGN_P9_A', 'SGN_P-8_A', 'SGN_P-2_C', 'SGN_P0_G', 'SGN_P-17_C', 'SGN_P-18_T', 'Context_P1_T', 'SGN_P-7_A', 'Context_P1_A', 'SGN_P2_C', 'SGN_P-15_G', 'Editing_P-16', 'SGN_P-24_A', 'SGN_P-29_G', 'SGN_P-15_C', 'SGN_P-10_A', 'SGN_P-16_T', 'SGN_P3_C', 'SGN_P-18_C', 'SGN_P-11_T', 'Editing_P-12', 'Context_P-2_A', 'SGN_P-27_G', 'SGN_P-10_C', 'SGN_P-20_C', 'SGN_P7_C', 'SGN_P-19_C', 'Editing_P-18', 'SGN_P-28_C','SGN_P-30_A', 'SGN_P-29_C', 'SGN_P-26_A', 'SGN_P-20_G', 'SGN_P-30_G', 'Editing_P-11', 'SGN_P-9_G', 'SGN_P-14_C', 'SGN_P-28_T', 'SGN_mt_w_20', 'Editing_P-7', 'Editing_P-15', 'SGN_mt_w_12', 'SGN_P3_G', 'Context_P-3_A', 'SGN_P1_A', 'SGN_P9_G', 'SGN_P-18_G', 'SGN_P3_T', 'SGN_P-20_A', 'SGN_P-19_G', 'SGN_P-13_C', 'Context_P3_A', 'SGN_P-23_G', 'SGN_P-10_G', 'SGN_P-17_T', 'SGN_P-2_G', 'Context_P3_C', 'SGN_P-7_G', 'SGN_P-3_T', 'Context_P-2_T', 'SGN_P-2_A', 'Context_P3_T', 'Context_P2_G', 'SGN_P-11_G', 'SGN_P-4_G', 'SGN_P-9_C', 'SGN_P7_A', 'SGN_P-25_A', 'SGN_P-21_G', 'SGN_P-23_T', 'SGN_P7_G', 'SGN_P-6_G', 'SGN_P-15_A', 'SGN_P8_G', 'Context_P2_A', 'SGN_P-13_A', 'SGN_P-7_C', 'Context_P-2_C', 'SGN_P-9_T', 'SGN_P-18_A', 'Context_P-2_G', 'SGN_P0_A', 'SGN_P-22_A', 'Context_P-3_T', 'SGN_P-1_A', 'SGN_P-3_G', 'SGN_P-22_C', 'SGN_P-8_G', 'SGN_P-16_C', 'SGN_P-3_C', 'SGN_P-8_C', 'SGN_P-5_C', 'SGN_P-22_G', 'Context_P-3_C', 'SGN_P-26_G', 'Context_P2_T', 'SGN_P8_T', 'SGN_P-9_A', 'Context_P2_C', 'Context_P1_G', 'SGN_P-6_C', 'SGN_P-14_T', 'Context_P-1_C', 'SGN_P-24_G', 'SGN_P-6_A', 'SGN_P-14_A', 'SGN_P1_C','SGN_P-10_T', 'SGN_P-17_A', 'SGN_P-21_C', 'SGN_P-3_A', 'SGN_P-7_T', 'SGN_P-5_A', 'SGN_P7_T', 'SGN_mt_w_8', 'SGN_P-27_C', 'SGN_P9_T', 'SGN_P6_G', 'SGN_mt_w_28', 'SGN_P0_T', 'Context_P-1_G', 'Editing_P-14', 'SGN_P9_C', 'SGN_P-17_G', 'Context_P3_G', 'SGN_P-12_G', 'Context_P-3_G', 'SGN_P-6_T', 'SGN_P1_G', 'SGN_P6_A', 'SGN_P6_T', 'SGN_P-1_T', 'Context_P1_C', 'Context_P-1_T', 'SGN_P-11_A', 'Editing_P-10', 'SGN_P0_C', 'SGN_P-11_C', 'SGN_P-19_T', 'SGN_P-19_A', 'SGN_P2_G', 'Ccontent', 'SGN_P2_T', 'Acontent', 'Tcontent', 'Gcontent', 'SGN_Strand', 'GCcontent', 'Editing_Position']

In [17]:
def split_string(input_string, possible_substrings):
    found_substrings = []
    
    possible_substrings.sort(key=len, reverse=True)
    
    for substring in possible_substrings:
        pos = input_string.find(substring)
        if pos != -1:
            found_substrings.append((pos, substring))
            input_string = input_string.replace(substring, ' ' * len(substring), 1)

    found_substrings.sort()
    sorted_substrings = [substring for pos, substring in found_substrings]

    return sorted_substrings

In [18]:
split_string("paired_Context_P-1_G_Editing_P-3", feature_list)

['Context_P-1_G', 'Editing_P-3']

In [19]:
features_20_40 = feature_list[-40:-20]
features_0_20 = feature_list[0:-20]

In [21]:
df['x'] = df['column'].str.contains('Editing', na=False)
df.head()
df_no_editing = df[df['x'] == False]

In [23]:
df_no_editing

,column,Comparison,P-value,Combined P-value,Mean Efficiency,x
33,paired_SGN_P0_T_SGN_P-14_A,1 vs 0,8.993832e-09,4.138314e-17,"[(0, 0.10734182525433869), (1, 0.1978396516007...",False
34,paired_SGN_P0_T_SGN_P-14_A,1 vs 2,2.382968e-07,4.138314e-17,"[(0, 0.10734182525433869), (1, 0.1978396516007...",False
35,paired_SGN_P0_T_SGN_P-14_A,1 vs 3,1.850305e-05,4.138314e-17,"[(0, 0.10734182525433869), (1, 0.1978396516007...",False
45,paired_SGN_P-5_A_SGN_P-14_A,3 vs 0,1.797105e-07,4.732912e-16,"[(0, 0.11139192080378246), (1, 0.1269157657657...",False
46,paired_SGN_P-5_A_SGN_P-14_A,3 vs 1,7.031468e-04,4.732912e-16,"[(0, 0.11139192080378246), (1, 0.1269157657657...",False
...,...,...,...,...,...,...
1308,paired_SGN_P-21_C_SGN_P4_T,2 vs 0,4.772629e-02,4.772629e-02,"[(0, 0.10400674323634504), (2, 0.1425410819521...",False
1309,paired_SGN_P-21_C_SGN_P5_A,2 vs 0,4.772629e-02,4.772629e-02,"[(0, 0.10400674323634504), (2, 0.1425410819521...",False
1310,paired_SGN_P-21_C_SGN_P5_C,3 vs 1,4.772629e-02,4.772629e-02,"[(1, 0.10400674323634504), (3, 0.1425410819521...",False
1311,paired_SGN_P-21_C_SGN_P5_G,2 vs 0,4.772629e-02,4.772629e-02,"[(0, 0.10400674323634504), (2, 0.1425410819521...",False
